# Análise Exploratória - Vendas em uma Startup

### Kedro

In [1]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from kedro.io import DataCatalog
    catalog: DataCatalog

In [2]:
%load_ext kedro.ipython

[09/09/25 23:17:08] INFO     Using 'c:\Users\Zeus\Projetos\Projetos DS\Projetos                     __init__.py:270
                             Preditiva\otimizacao-de-ads-para-startup\.venv\Lib\site-packages\kedro                
                             \framework\project\rich_logging.yml' as logging configuration.                        

                    INFO     Registered line magic '%reload_kedro'                                   __init__.py:63

                    INFO     Registered line magic '%load_node'                                      __init__.py:65

                    INFO     Resolved project path as: c:\Users\Zeus\Projetos\Projetos DS\Projetos  __init__.py:180
                             Preditiva\otimizacao-de-ads-para-startup.                                             
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/09/25 23:17:09] INFO     Kedro is sending anonymous usage data with the sole purpose of improving plugin.py:243
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[09/09/25 23:17:10] INFO     Kedro project Otimizacao de Ads para Startup                           __init__.py:146

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:147
                             'pipelines'                                                                           

[09/09/25 23:17:11] INFO     Registered line magic 'run_viz'                                        __init__.py:153

### Importações de Bibliotecas

In [3]:

# Importando as bibliotecas necessárias
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


In [4]:

# Configurações
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

# Configurando para exibir todas as colunas
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)


## Visão inicial das bases disponíveis

#### Metadados

In [ ]:
df_campanha_metadados = catalog.load("metadados")
df_campanha_metadados

In [ ]:
# Aumentar o limite de largura da coluna para melhor visualização
pd.set_option("display.max_colwidth", None)
df_campanha_metadados

In [ ]:
# Voltando a visualização padrão
pd.set_option("display.max_colwidth", 50)

#### Campanha

In [14]:
df_campanha = catalog.load("campanhas")
df_campanha.head()

[09/09/25 23:18:55] INFO     Loading data from campanhas (ExcelDataset)...                     data_catalog.py:1046

,id_do_anuncio,campanha,faixa_etária,sexo,categoria_de_interesse,qte_de_impressões,qte_de_clicks,valor_investido_no_anúncio,Qte_de_Vendas_após_Clique
0,708746,Campanha A,30-34,M,15,7350,1,1.43,3
1,708749,Campanha A,30-34,M,16,17861,2,1.82,2
2,708771,Campanha A,30-34,M,20,693,0,0.00,0
3,708815,Campanha A,30-34,M,28,4259,1,1.25,1
4,708818,Campanha A,30-34,M,28,4133,1,1.29,2


In [ ]:
df_campanha = df_campanha.rename(columns={
    "qte_de_impressões": "impressoes",
    "qte_de_clicks": "cliques",
    "valor_investido_no_anúncio": "custo",
    "Qte_de_Vendas_após_Clique": "vendas"
})

In [ ]:
from unidecode import unidecode

df_campanha.columns = [unidecode(col) for col in df_campanha.columns]

In [ ]:
df_campanha.head()

#### Categorias

In [15]:
df_categorias = catalog.load("categorias")
df_categorias.head()

[09/09/25 23:18:57] INFO     Loading data from categorias (ExcelDataset)...                    data_catalog.py:1046

,categoria_de_interesse,descrição_da_categoria
0,2,Business and Industry
1,7,NaN
2,10,NaN
3,15,NaN
4,16,Entertainment


In [ ]:
df_categorias.columns = [unidecode(col) for col in df_categorias.columns]
df_categorias

Provável que estivessem usando celulas mescladas no excel, tendo texto apenas na primeira linha da mescla. Um ffill resolve isso.

#### Categoria Detalhada

In [ ]:
df_categoria_detalhada = catalog.load("categoria_detalhada")
df_categoria_detalhada.head(20)

- *Comportamento estranho das categorias*

A base de categoria parece ser mais abrangente e a Planilha 8 mais detalhada. Porém ambas tem diversos registros nulos e em Planilha8 não há uma referência de número da categoria.

Olhando pela planilha, na aba depara_categoria parece que Business and Industry se aplicaria às próximas 3 linhas também, que poderia ser usado um ffill pra preencher, mas assim teriamos diversos números de categorias de interesse com descrição repetida.

Enquanto se olharmos a planilha 8, dá pra ver que dentro de business industry temos 17 outras quebras, ou um tree map, onde as linhas sequenciadas representam subcategorias, mas que não se encaixaria nas 3 descrições nulas da aba de categorias.

### Decisão de como cruzar as categorias com suas descrições

In [ ]:
df_campanha["categoria_de_interesse"] = df_campanha["categoria_de_interesse"].astype("category")
df_campanha[["categoria_de_interesse"]].describe()

In [ ]:
df_categoria_detalhada.describe()

In [ ]:
df_categorias["descricao_da_categoria"].describe()

In [ ]:
df_categorias["descricao_da_categoria"].ffill().describe()

Dentre as categoiras presentes nas campanhas, são 40 valores únicos. Nas campanhas detalhadas da planilha8 existem 148 categorias e no df_cateogias são 9, porém realizando o preenchimento ffill ficariam 40 também, igual ao número presente nas campanhas, mas não seriam únicas.

Vou usar a df_categorias com ffill pra dar join com a df_campanhas

In [ ]:
df_categorias["descricao_da_categoria"] = df_categorias["descricao_da_categoria"].ffill()

In [ ]:
df_campanha_merged = (
    df_campanha.merge(
        df_categorias,
        how="left",
        on="categoria_de_interesse"
    )
)

In [7]:
import sys
sys.path.append("..")
from src.otimizacao_de_ads_para_startup.pipelines.data_processing.nodes import *


In [16]:

df_perfil = process_df_perfil(df_campanha, df_categorias)


## Análise Descritiva

In [19]:
df_perfil.to_csv("df_perfil.csv", index=False)

In [ ]:
# Ajustar a "categoria_de_interesse" pra categorica
df_campanha_merged["categoria_de_interesse"] = df_campanha_merged["categoria_de_interesse"].astype("category")
df_campanha_merged.info()

In [ ]:
df_campanha_merged.head()

In [ ]:
def full_describe(df) -> None:
    print("Info")
    print(df.info())
    print("\ndescribe de categóricas")
    display(df.describe(include=["object", "category"]))
    print("\ndescribe de numéricas")
    display(df.describe(include="number"))
    print("\nHead")
    display(df.head())

# Verificando valores únicos nas colunas categóricas
def value_counts(df) -> None:

    print("=== VALORES ÚNICOS NAS COLUNAS CATEGÓRICAS ===")

    for coluna in df.select_dtypes(include=["object", "category"]).columns:
        vc = df[coluna].value_counts().reset_index()
        vc.columns = [coluna, "qtd"]
        vc["percentual"] = (vc["qtd"] / vc["qtd"].sum() * 100).round(2)
        display(vc)


In [ ]:
full_describe(df_campanha_merged)

In [ ]:
value_counts(df_campanha_merged)

Como queremos identificar os melhores perfis pra uma nova campanha, acredito que seja melhor remover a coluna de campanha e id_do_anuncio pra agrupar tudo por perfil (faixa_etária, sexo e descrição_da_categoria).

Mas antes, quero validar se o desempenho dos perfis são parecidos em diferentes campanhas.

In [ ]:
df_campanha_merged["perfil"] = (
    df_campanha_merged["faixa_etaria"].astype(str) + " | " +
    df_campanha_merged["sexo"].astype(str) + " | " +
    df_campanha_merged["categoria_de_interesse"].astype(str)
)

Também precisamos de métricas melhores pra fazer as comparações. Usaremos algumas métricas padrões em marketing

In [ ]:
# Valor de venda do curso definido na apresentação do problema (disponível no Readme).
VALOR_VENDA = 85

# Calculando os KPIs
df_campanha_merged["faturamento"] = df_campanha_merged["vendas"] * VALOR_VENDA   # Receita total gerada pelas vendas
df_campanha_merged["lucro"] = df_campanha_merged["faturamento"] - df_campanha_merged["custo"]   # Lucro líquido = receita - custo do anúncio
df_campanha_merged["ctr"] = (df_campanha_merged["cliques"] / df_campanha_merged["impressoes"])   # Click Through Rate (%) = % de impressões que geraram clique
df_campanha_merged["tc"] = (df_campanha_merged["vendas"] / df_campanha_merged["cliques"]) # Taxa de conversão (%) = % de cliques que geraram venda
df_campanha_merged["cpc"] = df_campanha_merged["custo"] / df_campanha_merged["cliques"]   # Custo por clique médio
df_campanha_merged["cc"] = df_campanha_merged["custo"] / df_campanha_merged["vendas"]   # Custo por conversão (venda)
df_campanha_merged["roi"] = (df_campanha_merged["faturamento"] - df_campanha_merged["custo"]) / df_campanha_merged["custo"]   # Retorno sobre investimento (%) = lucro / custo * 100
df_campanha_merged["conv_i"] = df_campanha_merged["vendas"] / df_campanha_merged["impressoes"]  # Taxa de conversão por impressão

In [ ]:
df_campanha_merged.head()

In [ ]:
df_campanha_merged.sort_values(by="perfil", ascending=True).head(20)

Podemos identificar acima que existem valores diferentes pra um mesmo perfil dentro de uma mesma campanha. Isso pode acontecer por haver mais de um design usado pro mesmo perfil na mesma campanhas.
Além disso, os desempenhos de cada design tiveram diferenças relevantes. Caso tivessemos uma base que indicasse quais designes foram usados em quais id_de_anuncios, poderíamos explorar, além do perfil de usuário, qual design teve melhor desempenho.

Como não temos tais informações, agruparei o desempenho dos perfis, ignorando o id do anuncio e a campanha.

In [ ]:
df_perfil = (
    df_campanha_merged.groupby(
        ["faixa_etaria",
         "sexo",
         "categoria_de_interesse",
         "descricao_da_categoria",
         "perfil"],
         observed=True
    )
    .agg({
        "impressoes": "sum",
        "cliques": "sum",
        "custo": "sum",
        "vendas": "sum"}
    )
    .reset_index()
)

In [ ]:
df_perfil["faturamento"] = df_perfil["vendas"] * VALOR_VENDA   # Receita total gerada pelas vendas
df_perfil["lucro"] = df_perfil["faturamento"] - df_perfil["custo"]   # Lucro líquido = receita - custo do anúncio
df_perfil["ctr"] = (df_perfil["cliques"] / df_perfil["impressoes"])   # Click Through Rate (%) = % de impressões que geraram clique
df_perfil["tc"] = (df_perfil["vendas"] / df_perfil["cliques"]) # Taxa de conversão (%) = % de cliques que geraram venda
df_perfil["cpc"] = df_perfil["custo"] / df_perfil["cliques"]   # Custo por clique médio
df_perfil["cc"] = df_perfil["custo"] / df_perfil["vendas"]   # Custo por conversão (venda)
df_perfil["roi"] = (df_perfil["faturamento"] - df_perfil["custo"]) / df_perfil["custo"]   # Retorno sobre investimento (%) = lucro / custo * 100
df_perfil["conversao"] = df_perfil["vendas"] / df_perfil["impressoes"]  # Taxa de conversão por impressão

In [ ]:
full_describe(df_perfil)

Temos que nos atentar aos valores infinitos gerados em cc por conta da divisão por zero. Vou substituir por nulo 

In [ ]:
df_perfil["cc"] = df_perfil["cc"].replace([np.inf, -np.inf], np.nan)

In [ ]:
value_counts(df_perfil)

#### KPIs

In [ ]:
df_perfil.head()

Exportação

In [ ]:
# Identificando colunas quantitativas
colunas_quantitativas = df_campanha.select_dtypes(include=[np.number]).columns

# Criando histogramas
fig, axes = plt.subplots(2, 4, figsize=(20, 12))
axes = axes.ravel()

for i, coluna in enumerate(colunas_quantitativas):
    if i < 8:  # Limitando a 8 gráficos
        axes[i].hist(df_campanha[coluna], bins=20, alpha=0.7, edgecolor="black")
        axes[i].set_title(f"Distribuição de {coluna}")
        axes[i].set_xlabel(coluna)
        axes[i].set_ylabel("Frequência")
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:

print("=== MEDIDAS RESUMO DAS VARIÁVEIS QUANTITATIVAS ===")
medidas_resumo = df_campanha[colunas_quantitativas].describe()
display(medidas_resumo.round(2))

In [ ]:
# Matriz de correlação
correlacao = df_campanha[colunas_quantitativas].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlacao, annot=True, cmap="coolwarm", center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title("Matriz de Correlação entre Variáveis Quantitativas")
plt.tight_layout()
plt.show()

In [ ]:

print("\n=== CORRELAÇÕES MAIS FORTES ===")
# Encontrando as correlações mais fortes (excluindo correlação com ela mesma)
correlacoes = []
for i in range(len(correlacao.columns)):
    for j in range(i+1, len(correlacao.columns)):
        correlacoes.append((
            correlacao.columns[i],
            correlacao.columns[j],
            correlacao.iloc[i, j]
        ))

correlacoes_df = pd.DataFrame(correlacoes, columns=["Variável 1", "Variável 2", "Correlação"])
correlacoes_df = correlacoes_df.sort_values("Correlação", key=abs, ascending=False)
display(correlacoes_df.head(15).round(3))


In [ ]:

# Análise de ROI por segmentação
print("=== ANÁLISE DE ROI POR SEGMENTAÇÃO ===")

for coluna_qual in colunas_qualitativas:
    print(f"\nROI por {coluna_qual}:")
    roi_por_grupo = df_campanha.groupby(coluna_qual)["roi"].agg(["mean", "std", "count"]).round(2)
    roi_por_grupo.columns = ["ROI Médio (%)", "Desvio Padrão", "Quantidade"]
    print(roi_por_grupo)

    # Gráfico de boxplot
    plt.figure(figsize=(10, 6))
    df_campanha.boxplot(column="roi", by=coluna_qual, figsize=(10, 6))
    plt.title(f"Distribuição do ROI por {coluna_qual}")
    plt.suptitle("")  # Remove o título automático
    plt.show()


In [ ]:
# Calculando métricas do funil de vendas
print("=== FUNIL DE VENDAS GERAL ===")

total_impressoes = df_campanha["impressoes"].sum()
total_cliques = df_campanha["cliques"].sum()
total_vendas = df_campanha["vendas"].sum()
total_custo = df_campanha["custo"].sum()

ctr_geral = (total_cliques / total_impressoes) * 100
tc_geral = (total_vendas / total_cliques) * 100
roi_geral = ((total_vendas * VALOR_VENDA - total_custo) / total_custo) * 100

print(f"Total de Impressões: {total_impressoes:,}")
print(f"Total de Cliques: {total_cliques:,}")
print(f"Total de Vendas: {total_vendas:,}")
print(f"CTR Geral: {ctr_geral:.2f}%")
print(f"Taxa de Conversão Geral: {tc_geral:.2f}%")
print(f"ROI Geral: {roi_geral:.2f}%")
print(f"Custo Total: R$ {total_custo:,.2f}")
print(f"Faturamento Total: R$ {total_vendas * VALOR_VENDA:,.2f}")
print(f"Lucro Total: R$ {total_vendas * VALOR_VENDA - total_custo:,.2f}")

In [ ]:

# Visualizando o funil de vendas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras do funil
etapas = ["Impressões", "Cliques", "Vendas"]
valores = [total_impressoes, total_cliques, total_vendas]
percentuais = [100, ctr_geral, tc_geral]

ax1.bar(etapas, valores, color=["#FF6B6B", "#4ECDC4", "#45B7D1"])
ax1.set_title("Funil de Vendas - Valores Absolutos")
ax1.set_ylabel("Quantidade")
for i, v in enumerate(valores):
    ax1.text(i, v + max(valores)*0.01, f"{v:,}", ha="center", va="bottom")

# Gráfico de percentuais
ax2.bar(etapas, percentuais, color=["#FF6B6B", "#4ECDC4", "#45B7D1"])
ax2.set_title("Funil de Vendas - Percentuais")
ax2.set_ylabel("Percentual (%)")
for i, v in enumerate(percentuais):
    ax2.text(i, v + 1, f"{v:.2f}%", ha="center", va="bottom")

plt.tight_layout()
plt.show()


## Dados de Pesquisa

### Base Pesquisa

In [ ]:
# Dados da pesquisa de mercado
excel_pesquisa = pd.read_excel(caminho_dados / "dados_pesquisa.xlsx", sheet_name=None)
print("Resumo das abas encontradas:")
for nome, df in excel_pesquisa.items():
    print(f"{nome} | Shape: {df.shape}")


In [ ]:
df_pesquisa_metadados = excel_pesquisa["Metadados"]
df_pesquisa_metadados

In [ ]:
df_pesquisa = excel_pesquisa["dados_da_campanha"]
df_pesquisa

In [ ]:
df_pesquisa = df_pesquisa.set_index("id_do_respondente")
df_pesquisa

In [ ]:
full_describe(df_pesquisa)

In [ ]:
full_value_countes(df_pesquisa)

In [ ]:
# Análise do interesse no curso
print("=== ANÁLISE DO INTERESSE NO CURSO ===")

# Verificando se existe coluna de interesse
colunas_pesquisa = df_pesquisa.columns.tolist()
print(f"Colunas disponíveis: {colunas_pesquisa}")


In [ ]:
print(f"\nDistribuição de {"interesse_no_Curso"}:")
display(df_pesquisa["interesse_no_Curso"].value_counts())
print(f"Percentual de interessados: {(df_pesquisa["interesse_no_Curso"].value_counts(normalize=True) * 100).round(2)}")


In [ ]:

# Análise do preço que as pessoas pagariam
print("=== ANÁLISE DO PREÇO ===")

# Procurando por colunas relacionadas ao preço
colunas_preco = [col for col in colunas_pesquisa if "preco" in col.lower() or "valor" in col.lower() or "pagaria" in col.lower()]
print(f"Colunas de preço encontradas: {colunas_preco}")

if colunas_preco:
    for col in colunas_preco:
        print(f"\nEstatísticas de {col}:")
        print(df_pesquisa[col].describe())

        # Histograma do preço
        plt.figure(figsize=(10, 6))
        plt.hist(df_pesquisa[col].dropna(), bins=20, alpha=0.7, edgecolor="black")
        plt.title(f"Distribuição de {col}")
        plt.xlabel("Preço (R$)")
        plt.ylabel("Frequência")
        plt.grid(True, alpha=0.3)
        plt.show()

# 7.1 Análise do investimento necessário para dobrar as vendas
print("=== ANÁLISE PARA DOBRAR AS VENDAS ===")

vendas_atual = total_vendas
vendas_dobrar = vendas_atual * 2
custo_atual = total_custo

# Assumindo que a taxa de conversão se mantém
cliques_necessarios = vendas_dobrar / (tc_geral / 100)
custo_necessario = cliques_necessarios * (custo_atual / total_cliques)

print(f"Vendas atuais: {vendas_atual:,}")
print(f"Vendas desejadas: {vendas_dobrar:,}")
print(f"Cliques necessários: {cliques_necessarios:,.0f}")
print(f"Custo necessário: R$ {custo_necessario:,.2f}")
print(f"Investimento adicional necessário: R$ {custo_necessario - custo_atual:,.2f}")

# 7.2 Análise da relação entre investimento e vendas
print("=== RELAÇÃO ENTRE INVESTIMENTO E VENDAS ===")

# Correlação entre custo e vendas
correlacao_custo_vendas = df_campanha["custo"].corr(df_campanha["vendas"])
print(f"Correlação entre custo e vendas: {correlacao_custo_vendas:.3f}")

# Gráfico de dispersão
plt.figure(figsize=(10, 6))
plt.scatter(df_campanha["custo"], df_campanha["vendas"], alpha=0.6)
plt.xlabel("Custo da Campanha (R$)")
plt.ylabel("Vendas Realizadas")
plt.title("Relação entre Custo da Campanha e Vendas")
plt.grid(True, alpha=0.3)

# Linha de tendência
z = np.polyfit(df_campanha["custo"], df_campanha["vendas"], 1)
p = np.poly1d(z)
plt.plot(df_campanha["custo"], p(df_campanha["custo"]), "r--", alpha=0.8)

plt.show()

# Análise de desperdício (campanhas com baixa taxa de conversão)
print("\n=== ANÁLISE DE DESPERDÍCIO ===")
campanhas_baixa_conversao = df_campanha[df_campanha["tc"] < df_campanha["tc"].median()]
print(f"Campanhas com baixa conversão (< mediana): {len(campanhas_baixa_conversao)}")
print(f"Custo total em campanhas de baixa conversão: R$ {campanhas_baixa_conversao['custo'].sum():,.2f}")
print(f"Percentual do custo total: {(campanhas_baixa_conversao['custo'].sum() / total_custo * 100):.2f}%")

# 7.3 Análise de perfis com maior ROI
print("=== PERFIS COM MAIOR ROI ===")

# Top 10 campanhas com maior ROI
top_roi = df_campanha.nlargest(10, "roi")[["idade", "sexo", "interesses", "roi", "custo", "vendas"]]
print("Top 10 campanhas com maior ROI:")
print(top_roi.round(2))

# Análise por combinação de variáveis
if "idade" in df_campanha.columns and "sexo" in df_campanha.columns:
    print("\nROI por combinação de Idade e Sexo:")
    roi_idade_sexo = df_campanha.groupby(["idade", "sexo"])["roi"].agg(["mean", "count"]).round(2)
    roi_idade_sexo.columns = ["ROI Médio (%)", "Quantidade de Campanhas"]
    print(roi_idade_sexo.sort_values("ROI Médio (%)", ascending=False))

if "interesses" in df_campanha.columns:
    print("\nROI por Interesses:")
    roi_interesses = df_campanha.groupby("interesses")["roi"].agg(["mean", "count"]).round(2)
    roi_interesses.columns = ["ROI Médio (%)", "Quantidade de Campanhas"]
    print(roi_interesses.sort_values("ROI Médio (%)", ascending=False))

# Análise para alocação da verba
print("=== RECOMENDAÇÕES PARA ALOCAÇÃO DA VERBA ===")

VERBA_TOTAL = 20000

# Identificando os melhores perfis baseado no ROI
melhores_perfis = df_campanha.groupby(["idade", "sexo", "interesses"]).agg({
    "roi": "mean",
    "custo": "mean",
    "vendas": "mean",
    "ctr": "mean",
    "tc": "mean"
}).round(2)

melhores_perfis = melhores_perfis.sort_values("roi", ascending=False)
print("Melhores perfis baseado no ROI:")
print(melhores_perfis.head(10))

# Sugestão de alocação baseada no ROI
print(f"\n=== SUGESTÃO DE ALOCAÇÃO DA VERBA DE R$ {VERBA_TOTAL:,.2f} ===")

# Alocando mais verba para perfis com maior ROI
top_perfis = melhores_perfis.head(5)
total_roi_top = top_perfis["roi"].sum()

for i, (perfil, dados) in enumerate(top_perfis.iterrows()):
    # Alocação proporcional ao ROI
    alocacao = (dados["roi"] / total_roi_top) * VERBA_TOTAL
    print(f"\nPerfil {i+1}: {perfil}")
    print(f"  ROI médio: {dados['roi']:.2f}%")
    print(f"  Alocação sugerida: R$ {alocacao:,.2f}")
    print(f"  Vendas esperadas: {alocacao / dados['custo'] * dados['vendas']:.0f}")
    print(f"  Lucro esperado: R$ {alocacao / dados['custo'] * dados['vendas'] * VALOR_VENDA - alocacao:,.2f}")

print("\n=== ANÁLISE CONCLUÍDA ===")
print("Principais descobertas e recomendações foram apresentadas acima.")
print("Considere implementar as sugestões de alocação da verba para maximizar o ROI.")
